In [1]:
ALIVE = '*'
EMPTY = '_'

class Grid:
    def __init__(self, height, width):
        self.height = height 
        self.width = width 
        self.rows = []
        
        for _ in range(self.height):
            self.rows.append([EMPTY] * self.width)

    def get(self, y, x):
        return self.rows[y % self.height][x % self.width]
    
    def set(self, y, x, state):
        self.rows[y % self.height][x % self.width] = state 
    
    def __str__(self):
        for i in range(self.height):
            for j in range(self.width):
                print(self.rows[i][j], end='')
            print()
        return '' 

def count_neighbour(y, x, get):
    n_  = get(y -1 , x)
    ne = get(y-1, x + 1)
    e_ = get(y, x + 1)
    se = get(y + 1, x + 1)
    s_ = get(y + 1, x)
    sw = get(y + 1, x - 1)
    w_ = get(y, x-1)
    nw = get(y-1, x - 1)
    neighbour_states = [n_ ,ne, s_, se, s_, sw, w_, nw] 
    count = 0 
    for state in neighbour_states:
        if state == ALIVE:
            count += 1
    return count 

def game_logic(state, neighbour):
    if state == ALIVE:
        if neighbour < 2:
            return EMPTY
        elif neighbour > 3:
            return EMPTY
    else:
        if neighbour == 3:
            return ALIVE 
    return state

def step_cell(y, x, get, set):
    state = get(y, x)
    neighbours = count_neighbour(y, x, get)
    next_state = game_logic(state, neighbours)
    set(y, x, next_state)
    
def simulate(grid):
    next_grid = Grid(grid.height, grid.width)
    for y in range(grid.height):
        for x in range(grid.width):
            step_cell(y, x, grid.get, next_grid.set)
    return next_grid 

In [2]:
grid = Grid(5, 9)
grid.set(0, 3, ALIVE)
grid.set(1, 4, ALIVE)
grid.set(2, 2, ALIVE)
grid.set(2, 3, ALIVE)
grid.set(2, 4, ALIVE)
print(grid)


___*_____
____*____
__***____
_________
_________



In [3]:
for i in range(5):
    print(grid)
    grid = simulate(grid)

___*_____
____*____
__***____
_________
_________

____*____
_________
___**____
___*_____
_________

_________
_________
___**____
___**____
____*____

_________
___**____
___*_____
___*_*___
____*____

_________
___**____
_____*___
___*_____
____*____



In [4]:
from threading import Lock 


class LockingGrid(Grid):
    def __init__(self, height, width):
        super().__init__(height, width)
        self.lock = Lock()
    
    def __str__(self):
        with self.lock:
            return super().__str__()
    
    def get(self, y, x):
        with self.lock:
            return super().get(y, x)
    
    def set(self, y, x, state):
        with self.lock:
            return super().set(y, x, state)



In [5]:
from threading import Thread


def simulate_threaded(grid):
    next_grid = LockingGrid(grid.height, grid.weight)
    threads = []
    
    for y in range(grid.height):
        for x in range(grid.width):
            args = (y, x, grid.get, grid.set)
            thread = Thread(target=step_cell, args=args)
            thread.start() # Fan Out Process 
            threads.append(thread)
    
    for thread in threads:
        thread.join() # Fan in 
    
    return next_grid
        
grid = LockingGrid(5, 9)
grid.set(0, 3, ALIVE)
grid.set(1, 4, ALIVE)
grid.set(2, 2, ALIVE)
grid.set(2, 3, ALIVE)
grid.set(2, 4, ALIVE)
print(grid)


___*_____
____*____
__***____
_________
_________



In [6]:
for i in range(5):
    print(grid)
    grid = simulate(grid)

___*_____
____*____
__***____
_________
_________

____*____
_________
___**____
___*_____
_________

_________
_________
___**____
___**____
____*____

_________
___**____
___*_____
___*_*___
____*____

_________
___**____
_____*___
___*_____
____*____



NameError: name 'count' is not defined